In [30]:
import numpy as np
import pandas as pd
import hopsworks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [13]:
# Login to Hopsworks
project = hopsworks.login()

# Retrieve Feature Store
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Multiple projects found. 

	 (1) marco
	 (2) quickstart_shared



Enter project to access:  1



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/397461
Connected. Call `.close()` to terminate connection gracefully.


In [14]:
# Retrieve Feature Group
port_embeddings_fg = fs.get_or_create_feature_group(
    name='port_embeddings',
    version=1,
)

# Retrieve Feature Groups
port_distance_fg = fs.get_or_create_feature_group(
    name='port_distance',
    version=1,
)

In [15]:
query = port_embeddings_fg.select_all().join(port_distance_fg.select_all(), on=["source_node_id", "target_node_id"])

In [16]:
feature_view = fs.get_or_create_feature_view(
    name='port_fv',
    version=1,
    query=query,
    #labels=[""],
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/397461/fs/393284/fv/port_fv/version/1


In [ ]:
# Create a csv dataset from Hopsworks feature view
td_version, td_job = feature_view.create_training_data(
    description = 'Training dataset',
    data_format = 'csv',
    write_options = {'wait_for_job': True}
)

In [23]:
X_train, y_train, X_test, y_test = feature_view.train_test_split(test_size=0.2)

Finished: Reading data from Hopsworks, using ArrowFlight (2.26s) 


In [25]:
display(X_test)

None

In [22]:
display(label_df)

None

In [ ]:
# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential()
model.add(Dense(128, input_dim=256, activation='relu'))  # 128 neurons for each embedding, input_dim is the total number of features (2 * 128)
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))  # Output layer with a single neuron for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse}')